##### Copyright 2020 The TensorFlow Authors.

In [ ]:
#@title Licensed under the Apache License, Version 2.0 (the "License");
# you may not use this file except in compliance with the License.
# You may obtain a copy of the License at
#
# https://www.apache.org/licenses/LICENSE-2.0
#
# Unless required by applicable law or agreed to in writing, software
# distributed under the License is distributed on an "AS IS" BASIS,
# WITHOUT WARRANTIES OR CONDITIONS OF ANY KIND, either express or implied.
# See the License for the specific language governing permissions and
# limitations under the License.

# TensorFlow Recommenders: Quickstart

<table class="tfo-notebook-buttons" align="left">
  <td>
    <a target="_blank" href="https://www.tensorflow.org/recommenders/quickstart"><img src="https://www.tensorflow.org/images/tf_logo_32px.png" />View on TensorFlow.org</a>
  </td>
  <td>
    <a target="_blank" href="https://colab.research.google.com/github/tensorflow/recommenders/blob/main/docs/examples/quickstart.ipynb"><img src="https://www.tensorflow.org/images/colab_logo_32px.png" />Run in Google Colab</a>
  </td>
  <td>
    <a target="_blank" href="https://github.com/tensorflow/recommenders/blob/main/docs/examples/quickstart.ipynb"><img src="https://www.tensorflow.org/images/GitHub-Mark-32px.png" />View source on GitHub</a>
  </td>
  <td>
    <a href="https://storage.googleapis.com/tensorflow_docs/recommenders/docs/examples/quickstart.ipynb"><img src="https://www.tensorflow.org/images/download_logo_32px.png" />Download notebook</a>
  </td>
</table>

In this tutorial, we build a simple matrix factorization model using the [MovieLens 100K dataset](https://grouplens.org/datasets/movielens/100k/) with TFRS. We can use this model to recommend movies for a given user.

### Import TFRS

First, install and import TFRS:

In [ ]:
!pip install -q tensorflow-recommenders
!pip install -q --upgrade tensorflow-datasets

In [ ]:
from typing import Dict, Text

import numpy as np
import tensorflow as tf
import pandas as pd
import pprint

import tensorflow_datasets as tfds
import tensorflow_recommenders as tfrs

### Read the data Interaksi pengguna


In [ ]:
# Ratings data.
dataset_interaksi = 'dataset/Interaksi_Pengguna_dengan_Resep.csv'
df_ratings = pd.read_csv(dataset_interaksi)

def create_tf_dataset(dataframe):
    # Convert 'user_id' to string
    dataframe['user_id'] = dataframe['user_id'].astype(str)
    dataframe['food_name'] = dataframe['food_name'].astype(str)

    # Convert Pandas DataFrame to TensorFlow Dataset
    tf_dataset = tf.data.Dataset.from_tensor_slices({
        'user_id': tf.constant(dataframe['user_id'].values, dtype=tf.string),
        'food_name': tf.constant(dataframe['food_name'].values, dtype=tf.string),
        'rating': tf.constant(dataframe['rating'].values, dtype=tf.int32),
    })
    return tf_dataset

# Create TensorFlow Dataset for the entire dataset
ratings_food = create_tf_dataset(df_ratings)
ratings = ratings_food.map(lambda x: {
    "user_id": x["user_id"],
    "food_name": x["food_name"]
})
print(ratings_food)

In [ ]:
dataset_resep = 'Resep_Dengan_Kolom_Alergen.csv'
df_resep = pd.read_csv(dataset_resep)

df_resep.head()

In [ ]:
# Ratings data.
dataset_food = 'dataset/Resep_Dengan_Kolom_Alergen.csv'
df_ratings = pd.read_csv(dataset_food)

def create_tf_dataset(dataframe):
    # Convert 'user_id' to string
    dataframe['food_name'] = dataframe['food_name'].astype(str)

    # Convert Pandas DataFrame to TensorFlow Dataset
    tf_dataset = tf.data.Dataset.from_tensor_slices({
        'food_name': tf.constant(dataframe['food_name'].values, dtype=tf.string),

    })
    return tf_dataset

# Create TensorFlow Dataset for the entire dataset
food = create_tf_dataset(df_ratings)
food = food.map(lambda x: x["food_name"])
print(food)


Build vocabularies to convert user ids and movie titles into integer indices for embedding layers:

In [ ]:
user_ids_vocabulary = tf.keras.layers.StringLookup(mask_token=None)
user_ids_vocabulary.adapt(ratings.map(lambda x: x["user_id"]))

food_name_vocabulary = tf.keras.layers.StringLookup(mask_token=None)
food_name_vocabulary.adapt(food)

### Define a model for food recommendation

In [ ]:
class FoodLensModel(tfrs.Model):
  # We derive from a custom base class to help reduce boilerplate. Under the hood,
  # these are still plain Keras Models.

  def __init__(
      self,
      user_model: tf.keras.Model,
      food_model: tf.keras.Model,
      task: tfrs.tasks.Retrieval):
    super().__init__()

    # Set up user and food representations.
    self.user_model = user_model
    self.food_model = food_model

    # Set up a retrieval task.
    self.task = task

  def compute_loss(self, features: Dict[Text, tf.Tensor], training=False) -> tf.Tensor:
    # Define how the loss is computed.

    user_embeddings = self.user_model(features["user_id"])
    food_embeddings = self.food_model(features["food_name"])

    return self.task(user_embeddings, food_embeddings)

Define the two models and the retrieval task.

In [ ]:
# Define user and movie models.
user_model = tf.keras.Sequential([
    user_ids_vocabulary,
    tf.keras.layers.Embedding(user_ids_vocabulary.vocab_size(), 64)
])
food_model = tf.keras.Sequential([
    food_name_vocabulary,
    tf.keras.layers.Embedding(food_name_vocabulary.vocab_size(), 64)
])

# Define your objectives.
task = tfrs.tasks.Retrieval(metrics=tfrs.metrics.FactorizedTopK(
    food.batch(128).map(food_model)
  )
)


### Fit and evaluate it.

Create the model, train it, and generate predictions:



In [ ]:
# Create a retrieval model.
model = FoodLensModel(user_model, food_model, task)
model.compile(optimizer=tf.keras.optimizers.Adagrad(0.5))

# Train for 3 epochs.
model.fit(ratings.batch(4096), epochs=10)

# Use brute-force search to set up retrieval using the trained representations.
index = tfrs.layers.factorized_top_k.BruteForce(model.user_model)
index.index_from_dataset(
    food.batch(100).map(lambda food_name: (food_name, model.food_model(food_name))))

# Get some recommendations.
_, food_name = index(np.array(["42"]))
print(f"Top 3 recommendations for user 42: {food_name[0, :3]}")

In [ ]:
# Get some recommendations.
_, food_name = index(np.array(["42"]))
print(f"Top 3 recommendations for user 42: {food_name[0, :3]}")